In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import collections
import os 

import warnings
warnings.filterwarnings("ignore")

import xesmf as xe

from utils import _remove_leap_days, _convert_ds_longitude

from regridding import apply_weights
from core import SpatialDisaggregator

import dask.distributed as dd
import dask_kubernetes as dk
import dask
import rhg_compute_tools.kubernetes as rhgk

In [2]:
spatial_disaggregator = SpatialDisaggregator(var='temperature')

### This notebook implements the class-based spatial disaggregation workflow. Validation is done in a separate notebook. 

1. compute multi-decade daily climatologies of ERA-5 at obs-res and coarsen it to model-res (they, e.g. NASA-NEX, do not say how, we will do bilinear for consistency with later step) ++
2. subtract (or divide for precip) BC’ed model data at model-res from obs climo at model resolution to `fit` a scaling factor **
3. bilinearly interpolate “scaling factor” (using xESMF) from the model grid to the obs grid ++
4. `Predict` by adding or multiplying the scaling factor to the obs-res daily climatology **

- ** steps that take place within `scikit-downscale`
- ++ steps that are part of the recipe _outside_ of `scikit-downscale`

load test bias corrected output from global bias correction prototype notebook (BC'ed NASA GISS CMIP6 data)

In [3]:
workdir = '/gcs/rhg-data/climate/downscaled/workdir'
year = 1990

In [4]:
tmax_model = xr.open_dataset(os.path.join(workdir, 
                                          'global_bias_correction_scaling_test.nc'))

In [5]:
tmax_model = tmax_model.loc[dict(time=slice("%s-01-01" %str(year), "%s-12-31" %str(year)))]

In [6]:
tmax_obs = xr.open_dataset(os.path.join('/gcs/rhg-data/climate/source_data/GMFD/tmax', 
                                         'tmax_0p25_daily_1990-1990.nc')).squeeze(drop=True
                                          ).rename({'latitude': 'lat', 'longitude': 'lon'})

In [7]:
# standardize longitudes 
tmax_obs = _convert_ds_longitude(tmax_obs, lon_name='lon')

# remove leap days 
tmax_obs = _remove_leap_days(tmax_obs)

Load daily obs climatology 

In [8]:
climo_obs_fine = (xr.open_dataset(os.path.join(workdir, 'gmfd_test_climo.nc'))
                 .rename({'latitude': 'lat', 'longitude': 'lon'}))

### Interpolate obs climo: fine -> coarse 

In [9]:
%%time 
obs_to_mod_weights = workdir + '/obs_to_mod_bilinear_spatial_disagg.nc'
regridder_obs_to_mod = xe.Regridder(tmax_obs.isel(time=0), tmax_model.isel(time=0), 
                         'bilinear', filename=obs_to_mod_weights, reuse_weights=True)

Reuse existing file: /gcs/rhg-data/climate/downscaled/workdir/obs_to_mod_bilinear_spatial_disagg.nc
CPU times: user 40.9 ms, sys: 28.1 ms, total: 69.1 ms
Wall time: 239 ms


In [10]:
%%time
climo_obs_coarse_lazy = xr.map_blocks(apply_weights, regridder_obs_to_mod, 
                                args=[climo_obs_fine['tmax']])

CPU times: user 277 ms, sys: 3.32 s, total: 3.6 s
Wall time: 13.3 s


In [11]:
%%time 
climo_obs_coarse = climo_obs_coarse_lazy.compute()

CPU times: user 317 µs, sys: 239 µs, total: 556 µs
Wall time: 565 µs


### Compute scaling factor by subtracting for temperature, dividing for precip, the BC'ed model data at model-res from obs climo at model-res. 

In [12]:
sfc = spatial_disaggregator.fit(tmax_model, climo_obs_coarse, var_name='tasmax')

### Interpolate scaling factor: coarse (model grid) -> fine (obs grid)

In [13]:
%%time
mod_to_obs_weights = workdir + '/mod_to_obs_bilinear_spatial_disagg.nc'
regridder_mod_to_obs = xe.Regridder(tmax_model.isel(time=0), 
                                    tmax_obs.isel(time=0), 
                         'bilinear', filename=mod_to_obs_weights, reuse_weights=True)

Reuse existing file: /gcs/rhg-data/climate/downscaled/workdir/mod_to_obs_bilinear_spatial_disagg.nc
CPU times: user 71.1 ms, sys: 154 ms, total: 225 ms
Wall time: 1.11 s


In [14]:
%%time
sfc = sfc.drop('dayofyear')
sff_lazy = xr.map_blocks(apply_weights, regridder_mod_to_obs, 
                                args=[sfc])
sff = sff_lazy.compute()

CPU times: user 5.36 s, sys: 12.1 s, total: 17.4 s
Wall time: 17.4 s


### Add (or multiply for precip) the scaling factor to the obs-res daily climatology

In [15]:
ds_varname = 'scale_factor_fine'
sff_ds = sff.to_dataset(name=ds_varname)

In [16]:
mod_yr_downscaled = spatial_disaggregator.predict(sff_ds, climo_obs_fine, var_name=ds_varname)

In [17]:
filepath = workdir + '/spatial_disagg_prototype/%s_class_recipe.nc' %str(year)
mod_yr_downscaled.to_netcdf(filepath)
print("finished %s" %str(year))

finished 1990
